In [61]:
import numpy as np
import pandas as pd

from collections import Counter

pd.set_option('display.max_rows', None)

In [62]:
wapo = pd.read_csv('datasets/wapo.csv')
mpv = pd.read_csv('datasets/mpv.csv')
mcdc = pd.read_csv('datasets/mcdc.csv')

In [63]:
len(wapo), len(mpv), len(mcdc)

(7514, 10212, 2143)

In [64]:
wapo.columns
wapo = wapo[['id', 'name', 'date', 'age', 'gender', 'race', 'city', 'state']]
wapo = wapo.dropna()

In [65]:
mpv.columns
mpv = mpv[['name', 'age', 'gender', 'race', 'date', 'city', 'state']]
mpv = mpv.dropna()

In [66]:
mcdc.columns
mcdc = mcdc[['ID', 'Victim\'s name', 'Age', 'Gender', 
             'Incident Date (MM/DD/YYYY)', 'City', 'State', 'WaPo ID (If included in WaPo database)']]

rename_dict = {'Victim\'s name': 'name',
               'Age': 'age',
               'Gender': 'gender',
               'Incident Date (MM/DD/YYYY)': 'date',
               'City': 'city',
               'State': 'state',
               'WaPo ID (If included in WaPo database)': 'wapo_id'}

mcdc = mcdc.rename(columns=rename_dict)

In [67]:
no_date = mcdc.iloc[0]
mcdc = mcdc.drop(0)

mcdc['date'] = pd.to_datetime(mcdc['date'])
wapo['date'] = pd.to_datetime(wapo['date'])
mpv['date'] = pd.to_datetime(mpv['date'])

In [68]:
split_names = mcdc['name'].str.split()
mcdc['name'] = [name for name in mcdc['name'].str.split()]
mcdc['age'] = mcdc['age'].astype(int, errors='ignore')
mcdc['wapo_id'] = mcdc['wapo_id'].replace(np.nan, 0)
mcdc['wapo_id'] = mcdc['wapo_id'].astype(int, errors='ignore')

In [58]:
wapo = wapo[wapo['race'] == 'B']
wapo = wapo.rename(columns={'id': 'wapo_id'})
wapo['name'] = [name for name in wapo['name'].str.split()]
wapo['age'] = wapo['age'].astype(int)
wapo['gender'] = wapo['gender'].replace(['M', 'F'], ['Male', 'Female'])
wapo['gender'] = wapo['race'].replace()

mpv = mpv[(mpv['race'].isin(['Black', 'Unknown']))]
mpv['name'] = [name for name in mpv['name'].str.split()]
mpv['age'] = mpv['age'].astype(int)
mpv = mpv[(mpv['race'].isin(['Black', 'Unknown']))]

In [59]:
wapo.to_csv('wapo.csv')
mpv.to_csv('mpv.csv')

In [60]:
len(wapo), len(mpv), len(mcdc)

(1606, 2502, 2142)

In [41]:
'''all wapo ids in dataset match on name'''
wapo_ids = mcdc.merge(wapo, on='wapo_id', how='inner')
wapos = wapo_ids[wapo_ids.apply(lambda x: any(i in x['name_x'] for i in x['name_y']), axis=1)]

In [49]:
'''remove all wapo ids from mcdc'''
mcdc = mcdc[~mcdc['wapo_id'].isin(wapos['wapo_id'])]
wapo = wapo[~wapo['wapo_id'].isin(wapos['wapo_id'])]

KeyError: 'wapo_id'

In [ ]:
len(wapo), len(mpv), len(mcdc)

In [44]:
'''check mcdc names against mpv and wapo'''
mcdc['name'] = mcdc['name'].fillna('noname')
mcdc['name'] = mcdc['name'].apply(tuple)
mpv['name'] = mpv['name'].apply(tuple)
wapo['name'] = wapo['name'].apply(tuple)
mpv_matches = mcdc.merge(mpv, on='name', how='inner')
wapo_matches = mcdc.merge(wapo, on='name', how='inner')

In [45]:
'''filter out mpv_matches from mcdc and mpv'''
mcdc = mcdc[~mcdc['name'].isin(mpv_matches['name'])]
mcdc = mcdc[~mcdc['name'].isin(wapo_matches['name'])]
mpv = mpv[~mpv['name'].isin(mpv_matches['name'])]
wapo = wapo[~wapo['name'].isin(wapo_matches['name'])]

In [46]:
len(wapo), len(mpv), len(mcdc)

(357, 1667, 69)

In [17]:
'''match on date'''
mpv_date_matches = mcdc.merge(mpv, on='date', how='inner')
wapo_date_matches = mcdc.merge(wapo, on='date', how='inner')

In [18]:
'''check that name matches when date matches'''
wapo_dates = wapo_date_matches[wapo_date_matches.apply(lambda x: any(i in x['name_x'] 
                                                       for i in x['name_y']), axis=1)]
mpv_dates = mpv_date_matches[mpv_date_matches.apply(lambda x: any(i in x['name_x'] 
                                                       for i in x['name_y']), axis=1)]

In [894]:
'''remove matches from mcdc, wapo, mpv'''
mcdc = mcdc[~mcdc['date'].isin(mpv_dates['date'])]
mcdc = mcdc[~mcdc['date'].isin(wapo_dates['date'])]
mpv = mpv[~mpv['date'].isin(mpv_dates['date'])]
wapo = wapo[~wapo['date'].isin(wapo_dates['date'])]

In [895]:
len(wapo), len(mpv), len(mcdc)

(345, 1642, 50)